In [35]:
from os import *
from os.path import *
from itertools import chain
import os
from sklearn import tree
import pandas as pd
from numpy import *
from pandas import *
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import pytz
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
import itertools
import numpy as np
np.random.seed(42)

geyser_power = 2200
fridge_power = 150
results_dic = {}

classifiers_dict = {"SVM": svm.SVC(), "KNN": KNeighborsClassifier(n_neighbors=3),
                    "DT": tree.DecisionTreeClassifier(),"RF":RandomForestClassifier()}



df_power= pd.read_csv("/Users/nipunbatra/git/energylensplus-offline/energylenserver/data/meter/103/PowerMeter.csv", index_col=0)
df_light= pd.read_csv("/Users/nipunbatra/git/energylensplus-offline/energylenserver/data/meter/103/LightMeter.csv", index_col=0)

df_power.index = pd.to_datetime(df_power.index, unit='s')
df_power_copy = df_power.copy()
df_power = df_power.resample("1T")

df_light.index = pd.to_datetime(df_light.index, unit='s')
df_light_copy = df_light.copy()
df_light = df_light.resample("1T")


df = df_power+df_light
df = df["2015-02-06"]

In [36]:
d = pd.read_csv("/Users/nipunbatra/git/energylensplus-offline/ground_truth/103_353325066658901_presencelog.csv", index_col=0)
d.index = pd.to_datetime(d.index, unit='s')
d['location']=1
d = d.resample('1T')
d=d.fillna(0)
df["occupancy"]=d

In [37]:
df = df.between_time("06:00", "22:00").copy()

In [38]:
from nilmtk.feature_detectors.steady_states import find_steady_states, find_steady_states_transients
from nilmtk.disaggregate.hart_85 import Hart85
h = Hart85()

In [39]:
geyser_min = geyser_power - 200
geyser_max = geyser_power + 200
fridge_min = fridge_power-100
fridge_max = fridge_power+100


In [52]:
pred_index = pd.DatetimeIndex(start="2015-02-06", end="2015-02-07", freq='1s')
pred = pd.DataFrame({"occupancy":np.zeros(len(pred_index))}, index=pred_index)

In [41]:
df_data = df_power_copy
ss, tr = find_steady_states(df_data)


Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [42]:
tr.head()

,active transition
2015-02-01 10:03:29,2167.900824
2015-02-01 10:45:20,121.368215
2015-02-01 11:09:29,-103.627729
2015-02-01 11:31:06,81.970468
2015-02-01 11:59:30,-2210.311545


In [43]:
from copy import deepcopy

In [44]:
h.transients = deepcopy(tr['2015-02-06'])

In [45]:
pair_df = h.pair(buffer_size=20,
                  min_tolerance=100, percent_tolerance=0.035,
                  large_transition=1000)

In [46]:
pair_df

,T1 Time,T1 Active,T2 Time,T2 Active
0,2015-02-06 00:16:38,125.174074,2015-02-06 00:30:25,-142.125668
1,2015-02-06 00:47:52,2117.572144,2015-02-06 00:47:55,-2118.042867
2,2015-02-06 00:48:49,2115.641866,2015-02-06 00:48:57,-2117.186035
3,2015-02-06 00:49:27,2116.557818,2015-02-06 00:49:35,-2117.610504
4,2015-02-06 00:50:08,2118.930389,2015-02-06 00:50:11,-2118.809891
5,2015-02-06 00:51:04,2115.021692,2015-02-06 00:51:23,-2115.772186
6,2015-02-06 00:13:22,2273.560027,2015-02-06 01:08:10,-2203.388717
7,2015-02-06 01:29:21,139.560556,2015-02-06 01:46:36,-149.690234
8,2015-02-06 02:33:44,1777.986825,2015-02-06 02:37:33,-1792.095590
9,2015-02-06 02:42:46,123.635863,2015-02-06 02:56:38,-121.299202


In [17]:
%matplotlib qt

In [70]:
ax = df.plot(subplots=True)

In [57]:
for idx, row in pair_df.iterrows():
    start = row['T1 Time']
    end = row['T2 Time']
    if not((fridge_min<=row['T1 Active']<=fridge_max) or (geyser_min<=row['T1 Active']<=geyser_max)):
        print "-----Filling between----", row

        pred[start:end] = 1
    else:
        pass
        #print "********Fridge or geyser**********", row

-----Filling between---- T1 Time      2015-02-06 02:33:44
T1 Active               1777.987
T2 Time      2015-02-06 02:37:33
T2 Active              -1792.096
Name: 8, dtype: object
-----Filling between---- T1 Time      2015-02-06 02:25:49
T1 Active               1097.899
T2 Time      2015-02-06 03:26:46
T2 Active              -1073.609
Name: 10, dtype: object
-----Filling between---- T1 Time      2015-02-06 09:46:34
T1 Active               1766.433
T2 Time      2015-02-06 09:48:13
T2 Active              -1770.378
Name: 16, dtype: object
-----Filling between---- T1 Time      2015-02-06 12:09:00
T1 Active               1768.199
T2 Time      2015-02-06 12:11:46
T2 Active              -1796.201
Name: 18, dtype: object
-----Filling between---- T1 Time      2015-02-06 12:14:54
T1 Active               1773.251
T2 Time      2015-02-06 12:15:07
T2 Active              -1742.562
Name: 19, dtype: object
-----Filling between---- T1 Time      2015-02-06 13:48:04
T1 Active               1759.862
T2 Ti

In [58]:
out = pred.resample("15T", how="max")

In [59]:
out.plot(ax=ax[1])

In [60]:
df_data = df_light_copy
ss, tr = find_steady_states(df_data)



Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [61]:
h.transients = deepcopy(tr['2015-02-06'])

In [62]:
pair_df = h.pair(buffer_size=20,
                  min_tolerance=100, percent_tolerance=0.035,
                  large_transition=1000)

In [63]:
pair_df

,T1 Time,T1 Active,T2 Time,T2 Active
0,2015-02-06 09:59:40,70.219876,2015-02-06 10:23:07,-70.219876
1,2015-02-06 15:26:14,88.511444,2015-02-06 15:26:36,-91.178333
2,2015-02-06 15:41:56,93.491934,2015-02-06 15:42:31,-93.491934


In [65]:
for idx, row in pair_df.iterrows():
    start = row['T1 Time']
    end = row['T2 Time']
    pred[start:end] = 1

In [68]:
out = pred.resample("15T", how="max")

In [71]:
out.plot(ax=ax[1])

In [73]:
tr['2015-02-06']

,active transition
2015-02-06 09:59:40,70.219876
2015-02-06 10:23:07,-70.219876
2015-02-06 15:26:14,88.511444
2015-02-06 15:26:36,-91.178333
2015-02-06 15:41:56,93.491934
2015-02-06 15:42:31,-93.491934
2015-02-06 17:00:58,-102.586336


In [74]:
pair_df

,T1 Time,T1 Active,T2 Time,T2 Active
0,2015-02-06 09:59:40,70.219876,2015-02-06 10:23:07,-70.219876
1,2015-02-06 15:26:14,88.511444,2015-02-06 15:26:36,-91.178333
2,2015-02-06 15:41:56,93.491934,2015-02-06 15:42:31,-93.491934


In [76]:
ss_power, tr_power = find_steady_states(df_power_copy)
days_power = pd.DatetimeIndex(np.unique(tr_power.index.date))

ss_light, tr_light = find_steady_states(df_light_copy)
days_light = pd.DatetimeIndex(np.unique(tr_light.index.date))

days = np.union1d(days_light, days_power)

Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [77]:
len(days_power), len(days_light), len(days)

(12, 9, 12)

In [79]:
days_light

DatetimeIndex(['2015-02-02', '2015-02-03', '2015-02-04', '2015-02-06',
               '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10',
               '2015-02-11'],
              dtype='datetime64[ns]', freq=None, tz=None)

In [82]:
days = pd.DatetimeIndex(np.union1d(days_light, days_power))

In [84]:
for day in days:
    try:
        tr_day_light = tr_light[day.strftime("%Y-%m-%d")]
        
    except:
        pass

In [89]:
tr_day_light.index[-1]<tr_day_light.index[0]

False

In [90]:
tr_day_power = tr_power[day.strftime("%Y-%m-%d")].abs()

In [91]:
tr_day_power

,active transition
2015-02-12 00:13:18,2281.654360
2015-02-12 00:21:09,115.111132
2015-02-12 00:35:22,139.591468
2015-02-12 00:59:12,2217.946593


In [98]:
tr_day_power_non_bg = tr_day_power[(tr_day_power['active transition']<fridge_min) & (tr_day_power['active transition']>fridge_max)
        &(tr_day_power['active transition']<geyser_min)&(tr_day_power['active transition']>geyser_max)]

In [99]:
tr_day_power_non_bg

,active transition


In [95]:
fridge_max, fridge_min, g

(250, 50)